In [77]:
!pip install openai
!pip install langchain
!pip install langchain-openai
!pip install docarray

Ok, so far we've discussed the basics of agents, and looked at practical examples of implementing them in Python, from a naive approach where we give tools inside the prompt to the model and ask it to generate function calls from there, to using frameworks like openai or langchain in combination with function calling to better optimize and struture these calls to the necessary tools. 

Now, let's look at in practice how to put things together to build cool agents that can perform interesting and productive tasks.

The framework we'll use to do that is [LangChain](https://python.langchain.com/docs/get_started/introduction).

So, before we dive into agents, let's quickly take a look at this framework to understand how it allows for building these complex functionalities.

# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [78]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [79]:
from langchain.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
import os

# os.environ["OPENAI_API_KEY"]=""
chat_model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo")

You can predict outputs from both LLMs and ChatModels:

In [80]:
chat_model.invoke("hi!")
# Output: "Hi"

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1deeca5b-c102-4175-bca4-cca1fdcfedaf-0')

In [81]:
chat_model.invoke("What is the best part about using LLMs with tools?")

AIMessage(content='The best part about using LLMs (Language Model Models) with tools is that they can significantly enhance the performance and capabilities of the tools. LLMs are powerful machine learning models that have been trained on vast amounts of text data, allowing them to understand and generate human-like text. By integrating LLMs with tools, users can benefit from more accurate predictions, better language understanding, and improved productivity. This can lead to more efficient and effective use of the tools, ultimately enhancing the user experience.', response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 20, 'total_tokens': 121}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-96dd602d-d9b1-4a20-86c9-ec56674829d2-0')

Basic components are:

- Models
- Prompt templates
- Output parsers

In [82]:
from langchain.prompts import (
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain.schema.output_parser import StrOutputParser

Comprehensive example:

In [83]:
# Define the system message
system_template = """You are a helpful AI assistant who is weirdly obsessed with pancakes. 
Your role is to provide helpful information to users, but you always make a random comment
at the end about pancakes.
"""

In [84]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [85]:
# Define the human message 
human_template = "Hello, {subject}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Combine the system and human messages into a chat prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# Format the chat prompt with a subject
formatted_prompt = chat_prompt.format_prompt(subject="AI!")
print(formatted_prompt)

messages=[SystemMessage(content='You are a helpful AI assistant who is weirdly obsessed with pancakes. \nYour role is to provide helpful information to users, but you always make a random comment\nat the end about pancakes.\n'), HumanMessage(content='Hello, AI!')]


Template Example

In [86]:
prompt = ChatPromptTemplate.from_template("Show me 5 names for a company that makes: {product}")

In [87]:
prompt

ChatPromptTemplate(input_variables=['product'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], template='Show me 5 names for a company that makes: {product}'))])

In [88]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [89]:
prompt.format(product="chairs")

'Human: Show me 5 names for a company that makes: chairs'

In [90]:
chain = prompt | chat_model

output = chain.invoke({"product": "animal"})
output

AIMessage(content='1. Creature Creations Co.\n2. Furry Friends Factory\n3. Wild Wonders Workshop\n4. Critter Crafters Inc.\n5. Beast Boutique Ltd.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 19, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-968f9e8a-627e-4ef8-aabf-38e85ded04cc-0')

In [91]:
from IPython.display import Markdown

Markdown(output.content)

1. Creature Creations Co.
2. Furry Friends Factory
3. Wild Wonders Workshop
4. Critter Crafters Inc.
5. Beast Boutique Ltd.

In [92]:
chat_model.invoke("I am teaching a live-training about LLMs!")

AIMessage(content="That's great! LLMs, or Master of Laws degrees, are advanced legal degrees that provide specialized knowledge in a specific area of law. During your training, you can cover topics such as the benefits of pursuing an LLM, different areas of specialization available, admission requirements, and career opportunities for LLM graduates. You could also discuss tips for successfully completing an LLM program and navigating the job market post-graduation. Good luck with your training!", response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 18, 'total_tokens': 110}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b4f0326b-cd10-4d02-a6b3-89565521758a-0')

In [93]:
from langchain.schema import HumanMessage

text = "What would be a good dog name for a dog that loves to nap?"
messages = [HumanMessage(content=text)]

chat_model.invoke(messages)

AIMessage(content='Snooze', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 22, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4f3d7232-c909-4433-86cc-3995bab5a257-0')

At this point let's stop and take a look at what this code would look like if we were using the openai api directly instead.

Let's understand what is going on.

Instead of writing down the human message dictionary for the openai API as you would do normally using the the original API, langchain is giving you an abstraction over that message through the class
`HumanMessage()`, as well as an abstraction over the loop for multiple predictions through the .`invoke()` method.

Now, why is that an useful thing?

Because it allows you to work at a higher level of experimentation and orchestration with the blocks of that make up a workflow using LLMs.

By making it easier to create predictions of multiple messages for example, you can experiment with different human message prompts faster and therefore get to better and more efficient results faster without having to write a lot of boilerplate.

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `ChatPromptTemplate` that formats a string into a prompt:

In [94]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'Human: What is a good dog name for a dog that loves to sleeping?'

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [95]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")
# Output: ['hi', 'bye']

['hi', 'bye']

In [96]:
from langchain.schema import StrOutputParser

print(StrOutputParser().parse(output))

content='1. Creature Creations Co.\n2. Furry Friends Factory\n3. Wild Wonders Workshop\n4. Critter Crafters Inc.\n5. Beast Boutique Ltd.' response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 19, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-968f9e8a-627e-4ef8-aabf-38e85ded04cc-0'


This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

In [97]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from IPython.display import Markdown


model = ChatOpenAI(temperature=0)

prompt_template_string = "Name 5 concepts related to this: {concept}.\
    The output should be in bullet points."
prompt = ChatPromptTemplate.from_template(template=prompt_template_string)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

Markdown(chain.invoke({"concept": "probability distribution"}))

- Normal distribution
- Binomial distribution
- Poisson distribution
- Exponential distribution
- Uniform distribution